<a href="https://colab.research.google.com/github/DrAlexSanz/NLP-SPEC-C2/blob/master/W3/Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Language Models: Auto-Complete
In this assignment, you will build an auto-complete system. Auto-complete system is something you may see every day

When you google something, you often have suggestions to help you complete your search.
When you are writing an email, you get suggestions telling you possible endings to your sentence.
By the end of this assignment, you will develop a prototype of such a system.

A key building block for an auto-complete system is a language model. A language model assigns the probability to a sequence of words, in a way that more "likely" sequences receive higher scores. For example,

* "I have a pen" is expected to have a higher probability than "I am a pen" since the first one seems to be a more natural sentence in the real world.

You can take advantage of this probability calculation to develop an auto-complete system.
Suppose the user typed

* "I eat scrambled" Then you can find a word x such that "I eat scrambled x" receives the highest probability. If x = "eggs", the sentence would be "I eat scrambled eggs"

While a variety of language models have been developed, this assignment uses N-grams, a simple but powerful method for language modeling.

* N-grams are also used in machine translation and speech recognition.
Here are the steps of this assignment:

1. Load and preprocess data
2. Load and tokenize data.
3. Split the sentences into train and test sets.
4. Replace words with a low frequency by an unknown marker <unk>.
5. Develop N-gram based language models
  * Compute the count of n-grams from a given data set.
  * Estimate the conditional probability of a next word with k-smoothing.
6. Evaluate the N-gram models by computing the perplexity score.
7. Use your own model to suggest an upcoming word given your sentence.

In [10]:
import math
import random
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
nltk.data.path.append('.')

!wget https://raw.githubusercontent.com/DrAlexSanz/NLP-SPEC-C2/master/W3/en_US.twitter.txt

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
--2020-10-22 06:17:22--  https://raw.githubusercontent.com/DrAlexSanz/NLP-SPEC-C2/master/W3/en_US.twitter.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3341555 (3.2M) [text/plain]
Saving to: ‘en_US.twitter.txt.1’

en_US.twitter.txt.1 100%[===================>]   3.19M  --.-KB/s    in 0.09s   

2020-10-22 06:17:22 (33.9 MB/s) - ‘en_US.twitter.txt.1’ saved [3341555/3341555]



In [2]:
with open("en_US.twitter.txt", "r") as f:
    data = f.read()
print("Data type:", type(data))
print("Number of letters:", len(data))
print("First 300 letters of the data")
print("-------")
display(data[0:300])
print("-------")

print("Last 300 letters of the data")
print("-------")
display(data[-300:])
print("-------")

Data type: <class 'str'>
Number of letters: 3335477
First 300 letters of the data
-------


"How are you? Btw thanks for the RT. You gonna be in DC anytime soon? Love to see you. Been way, way too long.\nWhen you meet someone special... you'll know. Your heart will beat more rapidly and you'll smile for no reason.\nthey've decided its more fun if I don't.\nSo Tired D; Played Lazer Tag & Ran A "

-------
Last 300 letters of the data
-------


"ust had one a few weeks back....hopefully we will be back soon! wish you the best yo\nColombia is with an 'o'...“: We now ship to 4 countries in South America (fist pump). Please welcome Columbia to the Stunner Family”\n#GutsiestMovesYouCanMake Giving a cat a bath.\nCoffee after 5 was a TERRIBLE idea.\n"

-------


## Part 1.2 Pre-process the data
Preprocess this data with the following steps:

1. Split data into sentences using "\n" as the delimiter.
2. Split each sentence into tokens. Note that in this assignment we use "token" and "words" interchangeably.
3. Assign sentences into train or test sets.
4. Find tokens that appear at least N times in the training data.
5. Replace tokens that appear less than N times by <unk>

**Note: we omit validation data in this exercise.**

In real applications, we should hold a part of data as a validation set and use it to tune our training.
We skip this process for simplicity.

## Exercise 01
Split data into sentences.

In [3]:
def split_to_sentences(data):
    """
    Take the full corpus and split into sentences.
    Input:
    data: a str that I read from the file
    Output:
    List of sentences    
    """
    sent = data.split("\n")

    # Remove leading and trailing spaces

    sent = [s.strip() for s in sent]

    # Remove empty sentences

    sent = [s for s in sent if len(s) > 0]

    return sent

In [4]:
# Test

X = """
I have a pen.\nI have an apple. \nAh\nApple pen.\n
"""

y = split_to_sentences(X)

print(y)

['I have a pen.', 'I have an apple.', 'Ah', 'Apple pen.']


## Exercise 02
The next step is to tokenize sentences (split a sentence into a list of words).

* Convert all tokens into lower case so that words which are capitalized (for example, at the start of a sentence) in the original text are treated the same as the lowercase versions of the words.
* Append each tokenized list of words into a list of tokenized sentences.
* Use str.lower to convert strings to lowercase.
* Please use nltk.word_tokenize to split sentences into tokens.
* If you used str.split instead of nltk.word_tokenize, there are additional edge cases to handle, such as the punctuation (comma, period) that follows a word.

In [18]:
def tokenize_sentences(sentences):
    """
    Take the previously split sentences to tokenize them
    Input: List of sentences output from the first function
    Output: List of list of tokens    
    """

    # Make everything lowercase
    # tokenize each word

    tok_sent = []

    for sent in sentences:
        sent = sent.lower()
        tokenized = nltk.word_tokenize(sent)
        tok_sent.append(tokenized)

    return tok_sent



In [19]:
# Test

sentences = ["Sky is blue.", "Leaves are green.", "Roses are red."]

tok_test = tokenize_sentences(sentences)

print(tok_test)

[['sky', 'is', 'blue', '.'], ['leaves', 'are', 'green', '.'], ['roses', 'are', 'red', '.']]


## Exercise 03
Use the two functions that you have just implemented to get the tokenized data.

1. Split the data into sentences.
2. Tokenize those sentences.

In [20]:
def tokenize_data(data):
    """
    Take the data from the assignment and tokenize it
    Input: Data, a str read from the file
    Output: list of lists with tokenized data
    """

    sentences = split_to_sentences(data)
    tokens = tokenize_sentences(sentences)

    return tokens

In [21]:
# TEST
x = "Sky is blue.\nLeaves are green\nRoses are red."
tokenize_data(x)

[['sky', 'is', 'blue', '.'],
 ['leaves', 'are', 'green'],
 ['roses', 'are', 'red', '.']]

## Split into train and test sets.

In [23]:
tokenized_data = tokenize_data(data)

random.seed(13)

random.shuffle(tokenized_data)

train_size = 0.8

train_data = tokenized_data[0:int(train_size * len(tokenized_data))]
test_data = tokenized_data[int(train_size * len(tokenized_data)):]

print("The {} data is split in {} train and {} test examples".format(len(tokenized_data), len(train_data), len(test_data)))

print("The first train sample is:", train_data[0])
print("The first test sample is:", test_data[0])

The 47961 data is split in 38368 train and 9593 test examples
The first train sample is: ['i', 'miss', 'you', 'too', '!']
The first test sample is: ['haha', 'just', 'txt', 'me']


## Exercise 04
You won't use all the tokens (words) appearing in the data for training. Instead, you will use the more frequently used words.

You will focus on the words that appear at least N times in the data.
* First count how many times each word appears in the data.
* You will need a double for-loop, one for sentences and the other for tokens within a sentence.

* If you decide to import and use defaultdict, remember to cast the dictionary back to a regular 'dict' before returning it.

In [25]:
def count_words(tokenized_sentences):
    """
    Count how many times a word appears in the data
    Input:
    List of list of words
    Output:
    Dictionary where keys are words and values are the counts
    """
    word_counts = {}

    # Go through each sentence. Inside each sentence go through each word.
    for sent in tokenized_sentences:
        for w in sent:
          # If the word is not included yet, include it and count that I have seen it once.
            if w not in word_counts.keys():
                word_counts[w] = 1
          # If the word already exists, add 1.
            else:
                word_counts[w] += 1
    
    return word_counts


In [27]:
test_a = [['sky', 'is', 'blue', '.'],
                       ['leaves', 'are', 'green', '.'],
                       ['roses', 'are', 'red', '.']]

a = count_words(test_a)

print(a)

{'sky': 1, 'is': 1, 'blue': 1, '.': 3, 'leaves': 1, 'are': 2, 'green': 1, 'roses': 1, 'red': 1}


## Handling 'Out of Vocabulary' words
If your model is performing autocomplete, but encounters a word that it never saw during training, it won't have an input word to help it determine the next word to suggest. The model will not be able to predict the next word because there are no counts for the current word.

* This 'new' word is called an 'unknown word', or out of vocabulary (OOV) words.
* The percentage of unknown words in the test set is called the OOV rate.

To handle unknown words during prediction, use a special token to represent all unknown words 'unk'.

* Modify the training data so that it has some 'unknown' words to train on.
* Words to convert into "unknown" words are those that do not occur very frequently in the training set.
* Create a list of the most frequent words in the training set, called the closed vocabulary .
* Convert all the other words that are not part of the closed vocabulary to the token 'unk'.

## Exercise 05
You will now create a function that takes in a text document and a threshold 'count_threshold'.

Any word whose count is greater than or equal to the threshold 'count_threshold' is kept in the closed vocabulary.
used that you want to keep, returns the document containing only the word closed vocabulary and the word unk.

In [29]:
def get_words_with_nplus_freq(tokenized_sentences, count_threshold):
    """
    Keep only the words that appear frequently. I will replace the others by <UNK>

    input: the tokenized sentences (list of lists) and a number
    Output: Vocabulary list    
    """

    closed_vocab = []
    word_count = count_words(tokenized_sentences)

    for word, count in word_count.items():
        if count >= count_threshold:
            closed_vocab.append(word)

    return closed_vocab

In [31]:
# Test

a_sentences = [['sky', 'is', 'blue', '.'],
                       ['leaves', 'are', 'green', '.'],
                       ['roses', 'are', 'red', '.']]

a_vocab = get_words_with_nplus_freq(a_sentences, count_threshold = 2)
print(a_vocab)

['.', 'are']


## Exercise 06
The words that appear 'count_threshold' times or more are in the 'closed vocabulary.

* All other words are regarded as 'unknown'.
* Replace words not in the closed vocabulary with the token "<unk>".